# Eigenvalues of the dielectric matrix

We compute a few eigenvalues of the dielectric matrix ($q=0$, $ω=0$) iteratively.

In [1]:
using DFTK
using Plots
using KrylovKit
using Printf

# Calculation parameters
kgrid = [1, 1, 1]
Ecut = 5

# Silicon lattice
a = 10.26
lattice = a / 2 .* [[0 1 1.]; [1 0 1.]; [1 1 0.]]
Si = ElementPsp(:Si, psp=load_psp("hgh/lda/Si-q4"))
atoms     = [Si, Si]
positions = [ones(3)/8, -ones(3)/8]

# Compute the dielectric operator without symmetries
model  = model_LDA(lattice, atoms, positions, symmetries=false)
basis  = PlaneWaveBasis(model; Ecut, kgrid)
scfres = self_consistent_field(basis, tol=1e-8);

n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -7.234093349019                   -0.50    8.0         
  2   -7.250324639508       -1.79       -1.40    1.0   6.84ms
  3   -7.251140641656       -3.09       -2.08    1.0   6.79ms
  4   -7.251130884697   +   -5.01       -1.99    2.0   8.64ms
  5   -7.251332730573       -3.69       -2.66    1.0   60.1ms
  6   -7.251337929565       -5.28       -3.10    1.0   7.04ms
  7   -7.251338701570       -6.11       -3.54    1.0   6.97ms
  8   -7.251338783212       -7.09       -4.10    1.0   7.05ms
  9   -7.251338797837       -7.83       -4.44    2.0   8.56ms
 10   -7.251338798349       -9.29       -4.71    2.0   8.28ms
 11   -7.251338798653       -9.52       -5.20    1.0   7.56ms
 12   -7.251338798699      -10.33       -5.74    2.0   8.81ms
 13   -7.251338798704      -11.31       -6.26    2.0   8.61ms
 14   -7.251338798705      -12.53       -6.83    2.0   8.39ms
 15   -7.

Applying $ε^† ≔ (1- χ_0 K)$ …

In [2]:
function eps_fun(δρ)
    δV = apply_kernel(basis, δρ; ρ=scfres.ρ)
    χ0δV = apply_χ0(scfres, δV)
    δρ - χ0δV
end;

… eagerly diagonalizes the subspace matrix at each iteration

In [3]:
eigsolve(eps_fun, randn(size(scfres.ρ)), 5, :LM; eager=true, verbosity=3);

[ Info: Arnoldi iteration step 1: normres = 0.06809425222129582
[ Info: Arnoldi iteration step 2: normres = 0.2898712187427273
[ Info: Arnoldi iteration step 3: normres = 0.843734922906015
[ Info: Arnoldi iteration step 4: normres = 0.4639078095034705
[ Info: Arnoldi iteration step 5: normres = 0.6402579792339624
[ Info: Arnoldi schursolve in iter 1, krylovdim = 5: 0 values converged, normres = (1.44e-01, 7.96e-02, 5.51e-01, 2.80e-01, 2.72e-02)
[ Info: Arnoldi iteration step 6: normres = 0.23415855269531177
[ Info: Arnoldi schursolve in iter 1, krylovdim = 6: 0 values converged, normres = (2.82e-02, 1.45e-01, 1.23e-01, 1.03e-01, 5.43e-02)
[ Info: Arnoldi iteration step 7: normres = 0.0677280025666337
[ Info: Arnoldi schursolve in iter 1, krylovdim = 7: 0 values converged, normres = (9.13e-04, 8.97e-03, 5.89e-03, 3.46e-02, 5.20e-02)
[ Info: Arnoldi iteration step 8: normres = 0.10421931538142641
[ Info: Arnoldi schursolve in iter 1, krylovdim = 8: 0 values converged, normres = (4.07e-05